<a href="https://colab.research.google.com/github/rand0mizator/notebooks/blob/main/%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0%20%D0%BF%D1%80%D0%BE%20%D0%BF%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3%20%D1%8F%D0%B7%D1%8B%D0%BA%D0%B0%20%D0%B8%20%D0%B3%D0%B5%D1%80%D0%BE%D1%8F%20%D0%B8%D0%B7%20%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
!pip install langchain openai langchain-openai langchain-community -q

In [90]:
import re
import pandas as pd
from tqdm import tqdm
from getpass import getpass

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, TransformChain
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

## Если используете ключ от OpenAI, запустите эту ячейку 👇

In [91]:
import os
from langchain_openai import ChatOpenAI


# os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# Инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0)

KeyboardInterrupt: Interrupted by user

## Если используете ключ из курса, запустите эти ячейки 👇


In [92]:
!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

--2024-04-28 13:43:20--  https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10823 (11K) [text/plain]
Saving to: ‘utils.py.1’

utils.py.1          100%[===================>]  10.57K  --.-KB/s    in 0s      

2024-04-28 13:43:21 (76.3 MB/s) - ‘utils.py.1’ saved [10823/10823]



In [93]:
from utils import ChatOpenAI
from getpass import getpass

#course_api_key= "Введите ваш API ключ, полученный в боте курса"
course_api_key = getpass(prompt='Введите ваш API ключ, полученный в боте курса')

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите ваш API ключ, полученный в боте курса··········


## Задание 3.2.9 🤔 Кажется, это что-то на LLM-ском? 🧐

In [94]:
df = pd.read_csv("https://stepik.org/media/attachments/lesson/1110883/raw_texts.csv")
df.head()

,raw_text
0,"The sun was setting, casting long shadows over..."
1,"Le soleil se couchait, jetant de longues ombre..."
2,"El sol se estaba poniendo, proyectando largas ..."
3,"La ciudad estaba llena de vida, sus calles lle..."
4,"La ville était pleine de vie, ses rues remplie..."


Напишем функцию, которая очистит текст от ненужных символов: `¿, ¡, £`

In [95]:
def clean_text(inputs: dict) -> dict:
    text = inputs["text"]
    clean_txt = re.sub(r'[¿|¡|£]+', '', text)
    inputs['text'] = clean_txt
    return inputs

Будем просить у модели определять язык и имя главного персонажа и выдавать ответ в виде словаря. Для этого создадим `Output parser`, с которым вы уже познакомились в прошлых уроках.

In [96]:
# Определим схемы ответа
language_schema = ResponseSchema(name="language", description="Определи язык на котором написан текст. Запиши название этого языка на английском. Например, не Русский, а Russian. Не Deutsch, а German")

person_schema = ResponseSchema(name="main_character", description="Определи имя главного героя текста. Запиши имя этого персонажа так, как оно записано в тексте. Например Hans, Pierrot, Анна")

response_schemas = [language_schema, person_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


Напишем шаблон промпта со своим вопросом и инструкциями по форматированию ответа. Будем передавать в этот промпт сырой текст

In [97]:
template = """\
Из следующего текста извлеки информацию:

language: Определи язык на котором написан текст. Запиши название этого языка на английском. Например, не Русский, а Russian. Не Deutsch, а German

main_character: Определи имя главного героя текста. Запиши имя этого персонажа так, как оно записано в тексте. Например Hans, Pierrot, Анна

text: {text}

{format_instructions}
"""
prompt = PromptTemplate(
    input_variables=['text'],
    template=template,
)


Создадим цепочку с помощью `LCEL`

In [98]:
chain = prompt | llm | output_parser

In [99]:
result = []
for text in tqdm(df['raw_text']):
    cln_txt = clean_text({'text': text})
    inputs = {'text': cln_txt['text'], 'format_instructions': format_instructions}
    answer = chain.invoke(inputs)
    inputs.update(answer)
    result.append(inputs)
    print(result)
    # break # Для отладки. Уберите, когда убедитесь, что на одном примере работает

  8%|▊         | 1/13 [00:01<00:12,  1.05s/it]

[{'text': "The sun was setting, casting long shadows over the small town. John, a middle-aged man with a heart full of dreams, was sitting on the porch of his old house. His friends, Peter and Mary, were there with him, sharing stories of their youth. They were all from the same town, and their lives were intertwined in ways they could never have imagined. Peter, a burly man with a booming voice, was the town's blacksmith. He was a man of few words, but his actions spoke volumes. Mary, on the other hand, was the town's schoolteacher. She was a woman of great wisdom and kindness, always ready to lend a helping hand. As they sat there, reminiscing about the past, a sense of nostalgia washed over them. They remembered the days when they were young and full of hope, when the world seemed full of possibilities. But now, as they looked at the setting sun, they realized that those days were long gone. John, however, was not ready to give up. He still had dreams, dreams that he was determined 

 15%|█▌        | 2/13 [00:01<00:10,  1.08it/s]

[{'text': "The sun was setting, casting long shadows over the small town. John, a middle-aged man with a heart full of dreams, was sitting on the porch of his old house. His friends, Peter and Mary, were there with him, sharing stories of their youth. They were all from the same town, and their lives were intertwined in ways they could never have imagined. Peter, a burly man with a booming voice, was the town's blacksmith. He was a man of few words, but his actions spoke volumes. Mary, on the other hand, was the town's schoolteacher. She was a woman of great wisdom and kindness, always ready to lend a helping hand. As they sat there, reminiscing about the past, a sense of nostalgia washed over them. They remembered the days when they were young and full of hope, when the world seemed full of possibilities. But now, as they looked at the setting sun, they realized that those days were long gone. John, however, was not ready to give up. He still had dreams, dreams that he was determined 

 23%|██▎       | 3/13 [00:02<00:08,  1.15it/s]

...

[{'text': "The sun was setting, casting long shadows over the small town. John, a middle-aged man with a heart full of dreams, was sitting on the porch of his old house. His friends, Peter and Mary, were there with him, sharing stories of their youth. They were all from the same town, and their lives were intertwined in ways they could never have imagined. Peter, a burly man with a booming voice, was the town's blacksmith. He was a man of few words, but his actions spoke volumes. Mary, on the other hand, was the town's schoolteacher. She was a woman of great wisdom and kindness, always ready to lend a helping hand. As they sat there, reminiscing about the past, a sense of nostalgia washed over them. They remembered the days when they were young and full of hope, when the world seemed full of possibilities. But now, as they looked at the setting sun, they realized that those days were long gone. John, however, was not ready to give up. He still had dreams, dreams that he was determined 

 92%|█████████▏| 12/13 [00:10<00:00,  1.23it/s]

[{'text': "The sun was setting, casting long shadows over the small town. John, a middle-aged man with a heart full of dreams, was sitting on the porch of his old house. His friends, Peter and Mary, were there with him, sharing stories of their youth. They were all from the same town, and their lives were intertwined in ways they could never have imagined. Peter, a burly man with a booming voice, was the town's blacksmith. He was a man of few words, but his actions spoke volumes. Mary, on the other hand, was the town's schoolteacher. She was a woman of great wisdom and kindness, always ready to lend a helping hand. As they sat there, reminiscing about the past, a sense of nostalgia washed over them. They remembered the days when they were young and full of hope, when the world seemed full of possibilities. But now, as they looked at the setting sun, they realized that those days were long gone. John, however, was not ready to give up. He still had dreams, dreams that he was determined 

100%|██████████| 13/13 [00:10<00:00,  1.20it/s]

[{'text': "The sun was setting, casting long shadows over the small town. John, a middle-aged man with a heart full of dreams, was sitting on the porch of his old house. His friends, Peter and Mary, were there with him, sharing stories of their youth. They were all from the same town, and their lives were intertwined in ways they could never have imagined. Peter, a burly man with a booming voice, was the town's blacksmith. He was a man of few words, but his actions spoke volumes. Mary, on the other hand, was the town's schoolteacher. She was a woman of great wisdom and kindness, always ready to lend a helping hand. As they sat there, reminiscing about the past, a sense of nostalgia washed over them. They remembered the days when they were young and full of hope, when the world seemed full of possibilities. But now, as they looked at the setting sun, they realized that those days were long gone. John, however, was not ready to give up. He still had dreams, dreams that he was determined 

In [102]:
result_df = pd.DataFrame(result)
result_df.head()

,text,format_instructions,language,main_character
0,"The sun was setting, casting long shadows over...",The output should be a markdown code snippet f...,English,John
1,"Le soleil se couchait, jetant de longues ombre...",The output should be a markdown code snippet f...,French,Pierre
2,"El sol se estaba poniendo, proyectando largas ...",The output should be a markdown code snippet f...,Spanish,Carlos
3,"La ciudad estaba llena de vida, sus calles lle...",The output should be a markdown code snippet f...,Spanish,Juan
4,"La ville était pleine de vie, ses rues remplie...",The output should be a markdown code snippet f...,French,Jean


Сохраним всё в итоговый файл. Убедитесь, что на этом этапе у вас в столбцах

- `text` - очищенный текст (без символов ¿, ¡, £)
- `language` - язык, на котором написан текст (название языка указать на английском языке)
- `main_character` - имя главного персонажа в тексте (указать на том языке, на котором и написан сам текст)

In [103]:
result_df[['text', 'language', 'main_character']].to_csv('3.2.9_solution.csv', index=False)